In [ ]:
# Cell 1: Imports
import os
import torch
from torchvision import models, transforms
from PIL import Image
import matplotlib.pyplot as plt
import json
from datetime import datetime

In [ ]:
def upscale_and_enhance(image):
    desired_size = (256, 256)  
    upscaled_image = image.resize(desired_size, Image.LANCZOS)
    return upscaled_image 

preprocess = transforms.Compose([
    transforms.Lambda(lambda img: upscale_and_enhance(img)),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

def preprocess_image(image_path):
    img = Image.open(image_path).convert('RGB')
    img_t = preprocess(img)
    return img_t



In [ ]:
# Cell 3: Load Model with Updated Method
from torchvision.models import resnet50, ResNet50_Weights

model = resnet50(weights=ResNet50_Weights.IMAGENET1K_V1)
model.eval()


In [ ]:
def predict_image_class(image_path, imagenet_labels):
    img = Image.open(image_path).convert('RGB')
    img_t = preprocess(img)
    batch_t = torch.unsqueeze(img_t, 0)

    with torch.no_grad():
        out = model(batch_t)
    probabilities = torch.nn.functional.softmax(out[0], dim=0)
    top5_prob, top5_catid = torch.topk(probabilities, 5)
    for i in range(top5_prob.size(0)):
        print(f"{i+1}: {imagenet_labels[top5_catid[i].item()]} - {top5_prob[i].item()}")



In [ ]:
# Cell 4: Modified to Display Images Alongside Predictions and Generate Classification Report
cropped_images_dir = 'cropped-objects'
imagenet_labels_json = 'imagenet-simple-labels.json'  

with open(imagenet_labels_json, 'r') as f:
    imagenet_labels = json.load(f)

def classify_image(image_path, model, preprocess, imagenet_labels, classification_counts, image_predictions):
    img = Image.open(image_path).convert('RGB')
    
    plt.imshow(img)
    plt.axis('off')  
    plt.show()

    img_t = preprocess(img)
    batch_t = torch.unsqueeze(img_t, 0)

    with torch.no_grad():
        out = model(batch_t)
    probabilities = torch.nn.functional.softmax(out[0], dim=0)
    top1_prob, top1_catid = torch.topk(probabilities, 1)
    class_label = imagenet_labels[top1_catid[0].item()]
    print(f"Image: {os.path.basename(image_path)}")
    print(f"Predicted Class: {class_label} - {top1_prob[0].item()}")
    classification_counts[class_label] = classification_counts.get(class_label, 0) + 1
    image_predictions[os.path.basename(image_path)] = (class_label, top1_prob[0].item())

    return classification_counts, image_predictions

classification_counts = {}
image_predictions = {}
total_images = 0

for image_file in os.listdir(cropped_images_dir):
    image_path = os.path.join(cropped_images_dir, image_file)
    classification_counts, image_predictions = classify_image(image_path, model, preprocess, imagenet_labels, classification_counts, image_predictions)
    total_images += 1

report_lines = [
    "Image Classification Report",
    "===========================",
    "",
    f"Date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}",
    "",
    f"Total Images Viewed: {total_images}",
    "",
    "Classification Counts:",
]

for class_label, count in sorted(classification_counts.items()):
    report_lines.append(f"{class_label}: {count}")

report_lines.append("")
report_lines.append("Image Predictions:")

for image_name, (class_label, confidence_score) in image_predictions.items():
    report_lines.append(f"{image_name}")
    report_lines.append(f"  Predicted Class: {class_label}")
    report_lines.append(f"  Confidence Score: {confidence_score:.4f}")
    report_lines.append("")

report_content = "\n".join(report_lines)

output_dir = "output"
os.makedirs(output_dir, exist_ok=True)

timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
timestamped_dir = os.path.join(output_dir, timestamp)
os.makedirs(timestamped_dir, exist_ok=True)

report_file = os.path.join(timestamped_dir, "classification-report.txt")
with open(report_file, "w") as f:
    f.write(report_content)

print(f"Classification report saved to {report_file}")